This Notebook will mainly used for my data science coursera course capstone project provided by IBM.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


Import the library

In [2]:
import requests
from bs4 import BeautifulSoup

Get the HTML page from the wikipedia url "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" and then parse it as a BeautifulSoup object

In [3]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
result = BeautifulSoup(page.content, "html.parser")

Get the table from the html

In [4]:
table_html = result.find("table")

Find all the column names in a tag "th" using find_all method

In [5]:
column_names_html = list(table_html.find_all("th"))

get the text from each element column_names_html and append it to a new list

In [6]:
column_names = []
for data in column_names_html:
    column_names.append(data.get_text())

Show the result column names

In [7]:
column_names

['Postcode', 'Borough', 'Neighbourhood\n']

Remove all the "\n"'s from each column names

In [8]:
column_names = [data.replace("\n", "") for data in column_names]

In [9]:
column_names

['Postcode', 'Borough', 'Neighbourhood']

Get the row data from the html table by find all the tag "tr"

In [10]:
rows = list(table_html.find_all("tr"))
rows_content = [[]]
for data in rows:
    rows_content.append(data.find_all("td"))

In [11]:
del(rows_content[:2])

Remove all the "\n"'s from each data in each rows

In [12]:
row_data = [[i.get_text().replace("\n","") for i in data] for data in rows_content]

Make data frame using the column names and the row data

In [13]:
dataframe = pd.DataFrame(row_data, columns = column_names)
dataframe = dataframe.groupby(["Postcode", "Borough"])['Neighbourhood'].apply(','.join).reset_index()

Check if there is a null value in the data

In [14]:
dataframe.isnull().sum()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

Show the first 5 rows of the dataframe

In [15]:
dataframe.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn


In [16]:
import geocoder
lat_lng_coords = [None for _ in dataframe.Neighbourhood]
for i,j in enumerate(lat_lng_coords):
    while(j is None):
        g = geocoder.google('{}'.format(dataframe.Postcode[i]))
        lat_lng_coords.append(g.latlng)

KeyboardInterrupt: 

Failed using the geocoder, so i'm goint to download the Geospatial_data instead

In [17]:
file_geo = "https://cocl.us/Geospatial_data"
geo = pd.read_csv(file_geo)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
new_dataframe= pd.merge(dataframe, geo, left_on = "Postcode", right_on = "Postal Code", how="inner")
new_dataframe.head(10)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


In [19]:
cleaned_dataframe = new_dataframe.drop(["Postal Code"], axis = 1)

In [20]:
cleaned_dataframe.isnull().sum()

Postcode         0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [21]:
cleaned_dataframe.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Cluster Toronto Neighbourhood

Import the library

In [26]:
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/putra/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          45 KB

The following packages will be DOWNGRADED:

  folium                                        0.10.0-py_0 --> 0.5.0-py_0



folium-0.5.0         | 45 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Visualize the the neigbourhood

In [36]:
map_ = folium.Map([43.806686, -79.194353], zoom_start = 11)
labels = [str(cleaned_dataframe.loc[i,"Postcode"])+", "+str(cleaned_dataframe.loc[i,'Neighbourhood']) for i in range(len(cleaned_dataframe['Neighbourhood']))]
for lat, lng, label in zip(cleaned_dataframe['Latitude'], cleaned_dataframe['Longitude'], labels):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_)
map_

Cluster the Neighbourhood

In [43]:
clusterNumber = 5
X = cleaned_dataframe[['Latitude', 'Longitude']]
k_means = KMeans(init = "k-means++", n_clusters = clusterNumber, n_init = 12)
k_means.fit(X)
class_label = k_means.labels_
class_label

array([2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 4, 4, 4,
       4, 4, 4, 0, 0, 0, 4, 4, 4, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3,
       4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3,
       3, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Add class label column on the dataframe

In [44]:
cleaned_dataframe['class_label'] = class_label

In [45]:
cleaned_dataframe.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,class_label
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2


Visualize the result of the clustering

In [46]:
# create map
map_clusters = folium.Map([43.806686, -79.194353], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(clusterNumber)
ys = [i + x + (i*x)**2 for i in range(clusterNumber)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cleaned_dataframe['Latitude'], cleaned_dataframe['Longitude'], labels, cleaned_dataframe['class_label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters